In [ ]:
# imports
import numpy as np
from typing import List, Callable
from scipy.optimize import minimize
from scipy.optimize._optimize import OptimizeResult
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp
from qiskit.primitives import StatevectorSampler, PrimitiveJob
from qiskit.circuit.library import TwoLocal
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_aer import AerSimulator

In [ ]:
# Setup the grader
from qc_grader.challenges.iqc_2024 import (
    grade_lab1_ex1,
    grade_lab1_ex2,
    grade_lab1_ex3,
    grade_lab1_ex4,
    grade_lab1_ex5,
    grade_lab1_ex6,
    grade_lab1_ex7,
)

In [ ]:
# Build a circuit to form a psi-minus Bell state
# Apply gates to the provided QuantumCircuit, qc
qbit1 = QuantumRegister(1)
qbit2 = QuantumRegister(1)
qc = QuantumCircuit(qbit1, qbit2)

qc.h(qbit1)
qc.cx(qbit1, qbit2)
qc.z(qbit1)
qc.x(qbit2)
### Write your code below here ###



### Don't change any code past this line ###
qc.measure_all()
qc.draw('mpl')

In [ ]:
%set_env QXToken=


In [ ]:
!pip install git+https://github.com/qiskit-community/Quantum-Challenge-Grader.git

In [ ]:
grade_lab1_ex1(qc)

In [ ]:
qc.measure_all()

### Write your code below here ###
from qiskit_aer.primitives import Sampler


sampler = StatevectorSampler()
pub = (qc)
job_sampler =  sampler.run([pub], shots = 256)


### Don't change any code past this line ###

result_sampler = job_sampler.result()
counts_sampler = result_sampler[0].data.meas.get_counts()

print(counts_sampler)

In [ ]:
grade_lab1_ex2(job_sampler)

In [ ]:
plot_histogram(counts_sampler)


In [ ]:
qc = QuantumCircuit(3)
qc.ry(1.91063324, 0)
qc.ch(0,1)
qc.cx(1,2)
qc.cx(0,1)
qc.x(0)
qc.measure_all()
qc.draw('mpl')

In [ ]:
grade_lab1_ex3(qc)

In [ ]:
sampler = StatevectorSampler()
pub = (qc)
job_sampler = sampler.run([pub], shots=10000)

result_sampler = job_sampler.result()
counts_sampler = result_sampler[0].data.meas.get_counts()

print(counts_sampler)
plot_histogram(counts_sampler)

In [ ]:
pauli_op = SparsePauliOp(['ZII', 'IZI', 'IIZ'])
print(pauli_op.to_matrix())

In [ ]:
num_qubits = 3
rotation_blocks = ['ry', 'rz']
entanglement_blocks = 'cz'
entanglement = 'full'

ansatz = TwoLocal(
    num_qubits=num_qubits,
    rotation_blocks=rotation_blocks,
    entanglement_blocks=entanglement_blocks,
    entanglement=entanglement,
    reps=1,  # Number of repetition of the rotation+entanglement blocks
    parameter_prefix='theta'  # Parameter prefix for the rotation angles
)


### Don't change any code past this line ###
ansatz.decompose().draw('mpl')

In [ ]:
grade_lab1_ex4(num_qubits, rotation_blocks, entanglement_blocks, entanglement) # Expected result type: int, List[str], str, str


In [ ]:
num_params = ansatz.num_parameters
num_params

In [ ]:
from qiskit import transpile
backend_answer = FakeSherbrooke()
optimization_level_answer = 0
pm = generate_preset_pass_manager(backend=backend_answer,optimization_level=optimization_level_answer)
isa_circuit = pm.run(ansatz)

In [ ]:
grade_lab1_ex5(isa_circuit)

In [ ]:
isa_circuit.draw('mpl', idle_wires=False,)

In [ ]:
hamiltonian_isa = pauli_op.apply_layout(layout=isa_circuit.layout)


In [ ]:
def cost_func(params, ansatz, hamiltonian, estimator, callback_dict):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    energy = result[0].data.evs[0]

    callback_dict["iters"] += 1
    callback_dict["prev_vector"] = params
    callback_dict["cost_history"].append(energy)


### Don't change any code past this line ###
    print(energy)
    return energy, result

In [ ]:
grade_lab1_ex6(cost_func)

In [ ]:
callback_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

In [ ]:
x0 = 2 * np.pi * np.random.random(num_params)
x0

In [ ]:
### Select a Backend
## Use FakeSherbrooke to simulate with noise that matches closer to the real experiment. This will run slower.
## Use AerSimulator to simulate without noise to quickly iterate. This will run faster.

backend = FakeSherbrooke()
# backend = AerSimulator()

# ### Don't change any code past this line ###

# Here we have updated the cost function to return only the energy to be compatible with recent scipy versions (>=1.10)
def cost_func_2(*args, **kwargs):
    energy, result = cost_func(*args, **kwargs)
    return energy

with Session(backend=backend) as session:
    estimator = Estimator(session=session)

    res = minimize(
        cost_func_2,
        x0,
        args=(isa_circuit, hamiltonian_isa, estimator, callback_dict),
        method="cobyla",
        options={'maxiter': 30})

In [ ]:
grade_lab1_ex7(res) # Expected result type: OptimizeResult


In [ ]:
fig, ax = plt.subplots()
plt.plot(range(callback_dict["iters"]), callback_dict["cost_history"])
plt.xlabel("Energy")
plt.ylabel("Cost")
plt.draw()